# Containerizer service

Import libraries that have been installed before

In [1]:
import sklearn
import modzymodel

### Train the model

This will train a sklearn model and it will be saved as a joblib file inside the `model` directory.

The goal for containerizer service is to create an image that exposes this model.

In [2]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from joblib import dump, load

digits = datasets.load_digits()

data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

dump(clf, './model/model.joblib')

['./model/model.joblib']

We check that the model has been correctly saved inside the `model` directory.

In [3]:
!ls ./model

model.joblib


### Define the values needed

The fields `name`, `version` and `api_key` are used when uploading the image to Modzy platform.

Since now we are just creating and downloading the docker image, the only fields that containerizer service actually needs are:

* `model_name`: name for the model inside the image
* `model_path`: directory that contains our model file

In [4]:
api_key = 'XXXX'

image_data = {
    'name': 'docker-registry:5000/sdk-test-sklearn',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': './model',
}

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: the name of the service that runs the containerizer

In [5]:
res = modzymodel.publish(
    api_key=api_key,
    module=sklearn,
    image_data=image_data,
    deploy=True,
    image_type=modzymodel.Constants.IMAGE_GREY,
    base_url='http://containerizer:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Publishing container... Ok!
Job ID: containerizer-builder-job-3b135c22-9b44-43c7-92de-07d6b8daf9f1


After the request is made, containerizer launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

In [7]:
modzymodel.get_job_status(job_id)

{'result': None,
 'status': {'active': None,
  'completion_time': 'Fri, 04 Jun 2021 12:16:17 GMT',
  'conditions': [{'last_probe_time': 'Fri, 04 Jun 2021 12:16:17 GMT',
    'last_transition_time': 'Fri, 04 Jun 2021 12:16:17 GMT',
    'message': None,
    'reason': None,
    'status': 'True',
    'type': 'Complete'}],
  'failed': None,
  'start_time': 'Fri, 04 Jun 2021 12:15:35 GMT',
  'succeeded': 1}}

Now, we should be able to see the created image listed in the registry. This means that the service has correctly created the image and uploaded it.

In [17]:
!wget -qO- http://docker-registry:5000/v2/_catalog

{"repositories":["sdk-test-sklearn"]}


### Download the tar file

Now that the job has finished, we can download the docker image that has been generated in the form of a tar file.

In [18]:
dst = './downloaded_image.tar'
modzymodel.download_tar(job_id, dst)

In [19]:
!ls -ltr ./

total 242152
drwsrwsr-x 2 jovyan users      4096 Jun  1 14:44 work
drwxr-sr-x 8 jovyan users      4096 Jun  4 12:14 containerizer
drwxr-sr-x 2 jovyan users      4096 Jun  4 12:15 model
-rw-r--r-- 1 jovyan users        38 Jun  4 12:17 _catalog
-rw-r--r-- 1 jovyan users      9317 Jun  4 12:17 demo.ipynb
-rw-r--r-- 1 jovyan users 247933952 Jun  4 12:18 downloaded_image.tar


In [20]:
from IPython.display import display, FileLink

local_file = FileLink(dst, result_html_prefix="Click here to download: ")
display(local_file)

/home/jovyan/downloaded_image.tar